<a href="https://colab.research.google.com/github/johnnykoo84/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

## 1. Load and clean the data

In [0]:
### YOUR CODE STARTS HERE

In [0]:
# imports
import pandas as pd
from scipy.stats import ttest_ind

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-30 03:13:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-01-30 03:13:14 (129 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

In [0]:
df = pd.read_csv('house-votes-84.data',
                 header=None,
                 names=column_headers,
                 na_values='?')

In [9]:
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [10]:
df.shape

(435, 17)

In [12]:
# recode votes as numeric
df = df.replace({'y': 1, 'n': 0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [13]:
# how did Republican vote?
rep = df[df['party'] == 'republican']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [14]:
# how did Democrats vote?
dem = df[df['party'] == 'democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


## 2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [0]:
import numpy as np

In [15]:
# the percentage of republicans who voted "yes" (1) 
# on the handicapped-infants bill
rep['handicapped-infants'].sum()/len(rep)


# len() is counting NaN values too!

0.18452380952380953

In [18]:
# Remove NaN values from this column
col = rep['handicapped-infants']
np.isnan(col)
hi_no_nans = col[~np.isnan(col)]
# The same column as before, but I've dropped the NaN values
hi_no_nans

0      0.0
1      0.0
7      0.0
8      0.0
10     0.0
      ... 
427    0.0
430    0.0
432    0.0
433    0.0
434    0.0
Name: handicapped-infants, Length: 165, dtype: float64

In [19]:
hi_no_nans.sum()/len(hi_no_nans)

0.18787878787878787

In [20]:
# Average rate of voting 'yes' on the handicapped-infants
rep['handicapped-infants'].mean()

0.18787878787878787

In [21]:
# let's do the same for Democrats
# Remove NaN values from this column
col = dem['handicapped-infants']
np.isnan(col)
hi_no_nans_dem = col[~np.isnan(col)]
# The same column as before, but I've dropped the NaN values
hi_no_nans_dem

3      0.0
4      1.0
5      0.0
6      0.0
9      1.0
      ... 
424    0.0
425    0.0
426    1.0
429    1.0
431    0.0
Name: handicapped-infants, Length: 258, dtype: float64

In [22]:
hi_no_nans_dem.sum()/len(hi_no_nans_dem)

0.6046511627906976

### Null Hypothesis 1) `handicapped-infants`
dem support `handicapped-infants` bill more than rep

In [23]:
# now let's do the hypothesis testing for 1)
ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'])

Ttest_indResult(statistic=nan, pvalue=nan)

In [24]:
# we have to drop NaN
ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

looking at the p-value of 1.61,
I failed to reject my hypothesis 1) dem support `handicapped-infants` bill
so this is not the one that I was looking for p < 0.1

## let's build a hypothesis for `water-project`

In [25]:
# mean support of Rep
rep['water-project'].mean()

0.5067567567567568

In [26]:
# mean support of Dem
dem['water-project'].mean()

0.502092050209205

I will take this hypothesis of goal 4 later

## let's build a hypothesis for `budget`

## let's build a hypothesis for `physician-fee-freeze`

## let's build a hypothesis for `el-salvador-aid`

## let's build a hypothesis for `religious-group`

## let's build a hypothesis for `anti-satellite-ban`

## let's build a hypothesis for `aid-to-contrast`

## let's build a hypothesis for `mx-missle`

## let's build a hypothesis for `immigration`

## let's build a hypothesis for `syn-fuels`

## let's build a hypothesis for `education`

## let's build a hypothesis for `mx-missle`

## let's build a hypothesis for `right-to-use`

## let's build a hypothesis for `crime`

## let's build a hypothesis for `duty-free`

## let's build a hypothesis for `south-africa`

### Null Hypothesis 2) `water-project`
dem support `water-project` bill more than rep

### Null Hypothesis 1) `handicapped-infants`
dem support `handicapped-infants` bill more than rep

### Null Hypothesis 1) `handicapped-infants`
dem support `handicapped-infants` bill more than rep

### Null Hypothesis 1) `handicapped-infants`
dem support `handicapped-infants` bill more than rep

### Null Hypothesis 1) `handicapped-infants`
dem support `handicapped-infants` bill more than rep

In [0]:
### Null Hypothesis 2) 

## 3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

## 4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)